In [1]:
import cv2
import numpy as np

In [2]:
def lucas_kanade(img1, img2, p0):
    if p0.shape[-2:] != (1, 2):
        print(f'p0.shape имеет не правильный формат, должно быть (-1, 1, 2), а он {p0.shape}')
        exit()

    lk_params = dict(
        winSize=(50,50),
        maxLevel=2,
        criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03),
    )

    #img1 = cv2.medianBlur(img1, 5)
    #img2 = cv2.medianBlur(img2, 5)
 
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    p1, st, err = cv2.calcOpticalFlowPyrLK(
        img1, img2, p0, None, **lk_params
    )

    # Пока не буду убирать пиксели которые не двигаются, так как возможно 
    # на других кадрах они подвинутся !!!
    # good_new = p1[st == 1]
    # good_old = p0[st == 1] 

    u = p1 - p0

    return p1, u

In [3]:
def construction_grid(a:int , b:int, n_x:int, n_y:int):
    # a - пикселей по y, b - пикселей по x, 
    # n_x - количество точек по x, n_y - количество точек по y

    points = np.zeros((n_x * n_y, 2), dtype='f')
    
    yw = (a % n_y) // 2
    step_y = a // n_y
    xw = (b % n_x) // 2
    step_x = b // n_x
    print(f'yw = {yw} \nxw = {xw} \nstep_y = {step_y} \nstep_x = {step_x}')

    k = 0

    for j in range(xw, b - xw, step_x):
        for i in range(yw, a - yw, step_y):
            # img[i][j] = black
            points[k] = [np.float32(j), np.float32(i)]
            k += 1
            
    points = points.reshape(points.shape[0], 1, points.shape[1])

    return points

In [4]:
import cv2
import numpy as np

name_video = '_Test1 - alecsandr27000.mp4'

cap = cv2.VideoCapture(name_video)

if not cap.isOpened():
    print("Ошибка открытия видеофайла")
    exit()

ret, frame_old = cap.read()
# ret, frame_new = cap.read()
a, b, r = frame_old.shape
p0 = construction_grid(a, b, 50, 50)
color = np.ones((100001, 3)) * 255

mask = np.zeros_like(frame_old)
# lucas_kanade(frame_old, frame_new, p0)
while True:
    ret, frame_new = cap.read()
    
    if not ret:
        break

    p1, ui = lucas_kanade(frame_old, frame_new, p0)

    for i, (new, old) in enumerate(zip(p1, p0)):
            a, b = new.ravel()
            c, d = old.ravel()
            mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color[i].tolist(), 1)
            frame_new = cv2.circle(frame_new, (int(a), int(b)), 3, color[i].tolist(), -1)

    img = cv2.add(frame_new, mask)
    cv2.imshow("frame", img)

    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

yw = 15 
xw = 10 
step_y = 21 
step_x = 38
